# Group 5
## Team members:
- Jianting Liu(8950907)
- David (8999846) 
- Marieth (9016702)


### Use case
The analysis examines the relationship between housing prices, income, and population in Canada over a 20-year period. By incorporating population data alongside existing housing prices and income metrics, we can better understand the demographic pressures on housing affordability.

## load data

In [1]:
import pandas as pd

data = pd.read_excel("./final_merged_data.xlsx")
data

,Date,Composite,One_storey,Two_storey,Townhouse,Apartment_unit,Median income,Population
0,2005-01-01,239800,207700,302900,201700,172000,4487.500000,1.069167e+07
1,2005-02-01,240500,208400,303300,202300,173000,4487.500000,1.069167e+07
2,2005-03-01,241300,209200,304300,202900,173900,4487.500000,1.069167e+07
3,2005-04-01,242000,210100,304800,203300,174600,4487.500000,1.071352e+07
4,2005-05-01,242600,210600,305400,203700,175400,4487.500000,1.071352e+07
...,...,...,...,...,...,...,...,...
210,2022-07-01,765100,644100,979100,709900,551500,5066.666667,1.297864e+07
211,2022-08-01,749600,630200,957700,693800,544100,5066.666667,1.297864e+07
212,2022-09-01,739200,619400,947700,684900,537100,5066.666667,1.297864e+07
213,2022-10-01,731400,613300,937000,679200,532400,5066.666667,1.309317e+07


## Converting factor variables to numeric


## Converting calendar dates to Julian


## Converting categorical values to dummies
 

## Performing Box-Cox transformations


## Applying Tukey's Ladder


## Perform SLR and MLR methods and analyze the results